In [1]:
from pycocotools.coco import COCO
import torch
# Effdet
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain, DetBenchPredict
from effdet.efficientdet import HeadNet
# WBF
from ensemble_boxes import *
import os
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import warnings
import math
import odach as oda
from itertools import product
warnings.filterwarnings(action='ignore') 

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [4]:
seed = 42

In [5]:
seed_everything(seed)

In [6]:
IMAGENET_DEFAULT_MEAN = [x * 255 for x in (0.485, 0.456, 0.406)]
IMAGENET_DEFAULT_STD = [x * 255 for x in (0.229, 0.224, 0.225)]

class CustomDataset(Dataset):
    '''
      data_dir: data가 존재하는 폴더 경로
      transforms: data transform (resize, crop, Totensor, etc,,,)
    '''
    def __init__(self, annotation, data_dir, transforms):
        super().__init__()
        self.data_dir = data_dir
        # coco annotation 불러오기 (coco API)
        self.coco = COCO(annotation)
        self.predictions = {
            "images": self.coco.dataset["images"].copy(),
            "categories": self.coco.dataset["categories"].copy(),
            "annotations": None
        }
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.coco.getImgIds(imgIds=index)
        image_info = self.coco.loadImgs(image_id)[0]
        
        image = cv2.imread(os.path.join(self.data_dir, image_info['file_name']))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image = (image - IMAGENET_DEFAULT_MEAN) / IMAGENET_DEFAULT_STD
        
        if self.transforms:
            sample = {
                'image': image,
            }
            sample = self.transforms(**sample)
            image = sample['image']
        
        return image, image_id
    
    def __len__(self) -> int:
        return len(self.coco.getImgIds())

In [7]:
def get_test_transform():
    return A.Compose([
        A.Resize(256,256),
        ToTensorV2(p=1.0)
    ])

In [8]:
test_annotation = '../input/data/test.json'
data_dir = '../input/data'
test_dataset = CustomDataset(test_annotation, data_dir, get_test_transform())
test_data_loader=DataLoader(test_dataset,batch_size=1,shuffle=False,num_workers=0)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


## Model Setting

In [9]:
def get_model():
    #tf_efficientdet_d5_ap
    config = get_efficientdet_config('tf_efficientdet_d4')
    config.image_size = (256, 256)
    config.norm_kwargs=dict(eps=.001, momentum=.01)
    net = EfficientDet(config, pretrained_backbone=False)
    #checkpoint = torch.load('./pretrained_weight/tf_efficientdet_d4_49-f56376d9.pth')
    #net.load_state_dict(checkpoint)

    net.reset_head(num_classes=11)
    net.class_net = HeadNet(config, num_outputs=11)
    
    model = DetBenchPredict(net)
    checkpoint = torch.load('./saved/checkpoint.pth')
    model.load_state_dict(checkpoint)
    return model.to(device)

In [10]:
model = get_model()
model.eval()

DetBenchPredict(
  (model): EfficientDet(
    (backbone): EfficientNetFeatures(
      (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act1): SwishMe()
      (blocks): Sequential(
        (0): Sequential(
          (0): DepthwiseSeparableConv(
            (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (act1): SwishMe()
            (se): SqueezeExcite(
              (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
              (act1): SwishMe()
              (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            )
            (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affi

## TTA

In [11]:
class BaseWheatTTA:
    """ author: @shonenkov """
    image_size = 256

    def augment(self, image):
        raise NotImplementedError
    
    def batch_augment(self, images):
        raise NotImplementedError
    
    def deaugment_boxes(self, boxes):
        raise NotImplementedError

class TTAHorizontalFlip(BaseWheatTTA):
    """ author: @shonenkov """

    def augment(self, image):
        return image.flip(1)
    
    def batch_augment(self, images):
        return images.flip(2)
    
    def deaugment_boxes(self, boxes):
        boxes[:, [1,3]] = self.image_size - boxes[:, [3,1]]
        return boxes

class TTAVerticalFlip(BaseWheatTTA):
    """ author: @shonenkov """
    
    def augment(self, image):
        return image.flip(2)
    
    def batch_augment(self, images):
        return images.flip(3)
    
    def deaugment_boxes(self, boxes):
        boxes[:, [0,2]] = self.image_size - boxes[:, [2,0]]
        return boxes
    
class TTARotate90(BaseWheatTTA):
    """ author: @shonenkov """
    
    def augment(self, image):
        return torch.rot90(image, 1, (1, 2))

    def batch_augment(self, images):
        return torch.rot90(images, 1, (2, 3))
    
    def deaugment_boxes(self, boxes):
        res_boxes = boxes.copy()
        res_boxes[:, [0,2]] = self.image_size - boxes[:, [3,1]] 
        res_boxes[:, [1,3]] = boxes[:, [0,2]]
        return res_boxes

class TTACompose(BaseWheatTTA):
    """ author: @shonenkov """
    def __init__(self, transforms):
        self.transforms = transforms
        
    def augment(self, image):
        for transform in self.transforms:
            image = transform.augment(image)
        return image
    
    def batch_augment(self, images):
        for transform in self.transforms:
            images = transform.batch_augment(images)
        return images
    
    def prepare_boxes(self, boxes):
        result_boxes = boxes.copy()
        result_boxes[:,0] = np.min(boxes[:, [0,2]], axis=1)
        result_boxes[:,2] = np.max(boxes[:, [0,2]], axis=1)
        result_boxes[:,1] = np.min(boxes[:, [1,3]], axis=1)
        result_boxes[:,3] = np.max(boxes[:, [1,3]], axis=1)
        return result_boxes
    
    def deaugment_boxes(self, boxes):
        for transform in self.transforms[::-1]:
            boxes = transform.deaugment_boxes(boxes)
        return self.prepare_boxes(boxes)

In [12]:
tta_transforms = []
for tta_combination in product([TTAHorizontalFlip(), None], 
                               [TTAVerticalFlip(), None],
                               [TTARotate90(), None]):
    tta_transforms.append(TTACompose([tta_transform for tta_transform in tta_combination if tta_transform]))

## Predict TTA -> WBF

In [14]:
def prediction_TTA(images):
    TTA_predictions = []
    with torch.no_grad():
        for tta_transform in tta_transforms:
            t_images = tta_transform.batch_augment(images.clone())
            det = model(t_images, None)
            predictions = []
            for i in range(t_images.shape[0]):
                boxes = det[i][:,:4]  
                scores = det[i][:,4]
                labels = det[i][:,5]

                npscore = scores.detach().cpu().numpy()
                indexes = np.where(npscore > 0)[0]
                boxes = boxes[indexes]
                boxes = boxes.clamp(0, 256)
                boxes = tta_transform.deaugment_boxes(boxes.copy())
                predictions.append({
                    'boxes': boxes[indexes],
                    'scores': scores[indexes],
                    "labels": labels[indexes],
                })
            TTA_predictions.append(predictions)
    return TTA_predictions

In [15]:
def run_wbf(predictions, image_index, image_size=256, iou_thr=0.5, skip_box_thr=0.005, weights=None):
    boxes = [(prediction[image_index]['boxes']/(image_size)).tolist() for prediction in predictions]
    scores = [prediction[image_index]['scores'].tolist() for prediction in predictions]
    labels = [prediction[image_index]['labels'].tolist() for prediction in predictions]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size)
    return boxes, scores, labels

## Inference

In [16]:
prediction_strings = []
file_names = []
coco = COCO(test_annotation)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [17]:
for i, (images, image_ids) in enumerate(tqdm(test_data_loader)):
    images = images.to(device).float()
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    
    predictions = prediction_TTA(images)
    
    # Must Batch = 1
    boxes, scores, labels = run_wbf(predictions, image_index=0)
    boxes = boxes * 2.
        
    prediction_string = ''
    for box, score, label in zip(boxes, scores, labels):
    #for box, score, label in zip(outputs['boxes'].tolist(), outputs['scores'].tolist(), outputs['labels'].tolist()):
        prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0]) + ' ' + str(
            box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + ' '
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])

  0%|          | 0/837 [00:00<?, ?it/s]


AttributeError: 'Tensor' object has no attribute 'copy'

In [25]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(f'effdet_TTA_submission.csv', index=None)
print(submission.head())

                                    PredictionString              image_id
0  8 0.845075786113739 177.93536376953125 232.656...  batch_01_vt/0021.jpg
1  8 0.701176106929779 286.1724853515625 183.3262...  batch_01_vt/0028.jpg
2  1 0.4273030757904053 236.6695556640625 461.791...  batch_01_vt/0031.jpg
3  8 0.6924664974212646 34.71138381958008 124.876...  batch_01_vt/0032.jpg
4  8 0.8632082343101501 151.97169494628906 0.0 35...  batch_01_vt/0070.jpg
